# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# for loading data
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

# for tokenizing
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

# for pipeline build
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

# for pipeline train
from sklearn.model_selection import train_test_split

# for model test
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# for gridcv
import numpy as np
from sklearn.preprocessing import LabelEncoder 
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\richa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\richa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\richa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///database_etl_rhn_udacity.db')
df = pd.read_sql("SELECT * FROM table_etl_rhn_udacity", engine)
X = df.message.values
y = df.iloc[:, 5:].values
categories = list(df.columns)[5:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):

    # tokenize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    tokens = word_tokenize(text)
        
    tokens = [t for t in tokens if t not in stopwords.words("english")] 
                 
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer() 

    # iterate through each token
    clean_tokens = []
    for tok in tokens:        
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        
        clean_tokens.append(clean_tok)

    return clean_tokens

In [4]:
# test out function
for message in X[:5]:
    tokens = tokenize(message)
    print(tokens)

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']
['is', 'hurricane']
['looking', 'someone', 'name']
['un', 'report', 'leogane', '80', '90', 'destroyed', 'only', 'hospital', 'st', 'croix', 'functioning', 'needs', 'supply', 'desperately']
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [7]:
# train classifier
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001D3CF9B8EE0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
y_pred = pd.DataFrame(pipeline.predict(X_test),columns=categories)
y_true=pd.DataFrame(y_test, columns=categories)
print(classification_report(y_true,y_pred,target_names=categories))

                        precision    recall  f1-score   support

               related       0.83      0.96      0.89      4948
               request       0.83      0.52      0.64      1033
                 offer       0.00      0.00      0.00        29
           aid_related       0.73      0.69      0.71      2604
          medical_help       0.63      0.07      0.12       520
      medical_products       0.73      0.12      0.20       307
     search_and_rescue       0.65      0.06      0.11       176
              security       0.00      0.00      0.00       114
              military       0.64      0.07      0.12       206
           child_alone       0.00      0.00      0.00         0
                 water       0.90      0.37      0.52       412
                  food       0.81      0.67      0.73       698
               shelter       0.84      0.42      0.56       548
              clothing       0.59      0.11      0.19        91
                 money       0.67      

C:\Users\richa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\richa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\richa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\richa\anaconda3\lib\site-packages\sklearn\metrics\_cla

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x000001D3CF9B8EE0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x000001D3CF9B8EE0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [10]:
parameters = {
    'vect__max_df': [0.5, 1.0],
    'vect__min_df': [1, 5],
    'tfidf__use_idf': [True, False],
    'clf__estimator__n_estimators': [10, 25],
    'clf__estimator__min_samples_split': [2, 4]
    }

In [11]:
# reduce the optimization time:
# instantiate gridsearchcv having cv=2 (2 fold validation, by default it is set to 5)
# and verbose = 3 to get the detailed realtime progress of the operation.
cv = GridSearchCV(pipeline, param_grid=parameters, cv = 2, verbose = 3)

In [12]:
def display_results(cv, y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test.values.argmax(axis=1),predictions.argmax(axis=1),labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)
    print("\nBest Parameters:", cv.best_params_)


def main():
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    cv.fit(X_train, y_train)
    y_pred = cv.predict(X_test)

    display_results(cv, y_test, y_pred)

main()

Fitting 2 folds for each of 32 candidates, totalling 64 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.5, vect__min_df=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.5, vect__min_df=1, score=0.232, total= 5.5min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.5, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.5min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.5, vect__min_df=1, score=0.240, total= 5.5min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.5, vect__min_df=5 


...

[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.5, vect__min_df=5, score=0.241, total= 5.3min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.5, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.5, vect__min_df=5, score=0.242, total= 5.3min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=1.0, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=1.0, vect__min_df=1, score=0.234, total= 5.4min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=1.0, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=1.0, vect__min_df=1, score

[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__max_df=1.0, vect__min_df=5, score=0.249, total= 5.5min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.5, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.5, vect__min_df=1, score=0.238, total= 5.3min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.5, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.5, vect__min_df=1, score=0.237, total= 5.3min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.5, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.5, vect__min_df=5, scor

[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__max_df=1.0, vect__min_df=1, score=0.245, total= 5.6min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__max_df=1.0, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__max_df=1.0, vect__min_df=1, score=0.242, total= 5.6min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__max_df=1.0, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__max_df=1.0, vect__min_df=5, score=0.253, total= 5.5min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__max_df=1.0, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__max_df=1.0, vect__min_df=5

[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed: 349.7min finished


ValueError: multilabel-indicator is not supported

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# Improved parameters 
parameters = {
    'vect__max_df': [0.5, 1.0],
    'vect__min_df': [1, 5],
    'tfidf__use_idf': [True, False],
    'clf__estimator__n_estimators': [10, 25],
    'clf__estimator__min_samples_split': [2, 4]
    }
# new model with improved parameters
cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=3)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.